## Minimize structures

Add missing hydrogen using pdbfixer. 5' base will be automatically detected so there is no need to change the first residue name.

**Notes**
Alternatively, missing hydrogens could be added by pdb4amber. Note that the first residue needs to be modify (e.g. A --> A5).
>import pdb4amber  
>pdb4amber.run(arg_pdbin='input.pdb', arg_pdbout='pdb4amber.pdb', arg_add_missing_atoms=True)

**References**
- [OpenMM-Tricks-and-Recipes](https://github-wiki-see.page/m/ParmEd/ParmEd/wiki/OpenMM-Tricks-and-Recipes)
- [OPENMM_TUTORIAL](https://gpantel.github.io/assets/PDF/OpenMM_Tutorial.pdf)  

In [1]:
import os, sys, shutil
import pathlib
import glob as glob
import numpy as np
import re
import warnings
import mdtraj as md
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
from openmm.app import PDBFile
from pdbfixer import PDBFixer

In [9]:
def minimize(output_path, file):
    
    """
    load file
    """
    fixer = PDBFixer(filename=file)
    fixer.findMissingResidues()
    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(True)
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    #fixer.addSolvent(fixer.topology.getUnitCellDimensions())
    #PDBFile.writeFile(fixer.topology, fixer.positions, open('pdbfixer.pdb', 'w'))
    
    
    """
    setup system
    """
    forcefield = ForceField('amber14/RNA.OL3.xml', 'implicit/gbn2.xml')
    system = forcefield.createSystem(fixer.topology, nonbondedMethod=NoCutoff, constraints=HBonds)
    
    # heavy atom restraint
    force = CustomExternalForce("k*((x-x0)^2+(y-y0)^2+(z-z0)^2)")
    force.addGlobalParameter("k", 5.0*kilocalories_per_mole/angstroms**2)
    force.addPerParticleParameter("x0")
    force.addPerParticleParameter("y0")
    force.addPerParticleParameter("z0")
    
    for i, atom in enumerate(fixer.topology.atoms()):
        if atom.element.symbol != "H":
            atom_crd = fixer.positions[i]
            force.addParticle(i, atom_crd.value_in_unit(nanometers))
    system.addForce(force)
    
    
    """
    minimize structure
    """
    integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
    simulation = Simulation(fixer.topology, system, integrator)
    simulation.context.setPositions(fixer.positions)
    simulation.minimizeEnergy(maxIterations=100)
    minpositions = simulation.context.getState(getPositions=True).getPositions()
    
    """
    save pdb
    """    
    try:
        basename = os.path.basename(file)
        PDBFile.writeFile(fixer.topology, minpositions, open(os.path.join(output_path, basename), 'w'))   
        
        # check if simulation can be run properly
        simulation.reporters.append(PDBReporter('/Users/takabak/Desktop/dump.pdb', 1))
        #simulation.reporters.append(StateDataReporter(stdout, 1, step=True, potentialEnergy=True, temperature=True))
        #simulation.reporters.append(StateDataReporter(stdout, 1, step=False, potentialEnergy=False, temperature=False))
        simulation.step(1)
    except:
        print("{}: Check structure!!")

In [10]:
def test(files):
    for f in files:
        fixer = PDBFixer(filename=f)
        fixer.findMissingResidues()
        fixer.findNonstandardResidues()
        fixer.findMissingAtoms()
        
        if fixer.missingAtoms:
            print("{}: missing atoms".format(f))
            shutil.move(f, f + ".warning")

In [11]:
if __name__ == "__main__":
    base_path = os.path.dirname(os.path.abspath("__file__")).strip('notebooks')
    output_path = os.path.join(base_path, "minimized")

    # motif
    _path = os.path.join(base_path, "pdb", "motif", "cluster", "triplebase")
    motif_files = glob.glob(_path + "/*/centroid/rep*.pdb")
    
    # triplebase
    _path = os.path.join(base_path, "pdb", "triplebase")
    triplebase_files = glob.glob(_path + "/*.pdb" )
        
    # basepairs
    _path = os.path.join(base_path, "pdb", "bpcatalog")
    bpcatalog_files = glob.glob(_path + "/*.pdb" )

    
    files = motif_files + triplebase_files + bpcatalog_files
    #files = triplebase_files
    print(">{} files found".format(len(files)))
    
    
    # check
    #test(files)
    
    
    for file in files:
        minimize(output_path, file)

>7632 files found


Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1


### simple test

In [ ]:
#f = files[0]
f = "/Users/takabak/work/rna_bgsu/pdb/triplebase/Triple_cSW_tHH_AGA.pdb"
#f = "/Users/takabak/sftp/Triple_cHH_cSS_GAA.pdb"
#f = "/Users/takabak/sftp/BP_cHH_AG.pdb"
#f = "/Users/takabak/sftp/HL_21000.1_AAA_6.pdb"

In [ ]:
fixer = PDBFixer(filename=f)
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
#fixer.addSolvent(fixer.topology.getUnitCellDimensions())
#PDBFile.writeFile(fixer.topology, fixer.positions, open('pdbfixer.pdb', 'w'))

In [ ]:
#pdb = PDBFile('pdb4amber.pdb')
#pdb = PDBFile('pdbfixer.pdb')

In [ ]:
#forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
forcefield = ForceField('amber14/RNA.OL3.xml', 'implicit/gbn2.xml')

In [ ]:
#system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=HBonds)
system = forcefield.createSystem(fixer.topology, nonbondedMethod=NoCutoff, constraints=HBonds)

In [ ]:
force = CustomExternalForce("k*((x-x0)^2+(y-y0)^2+(z-z0)^2)")
force.addGlobalParameter("k", 5.0*kilocalories_per_mole/angstroms**2)
force.addPerParticleParameter("x0")
force.addPerParticleParameter("y0")
force.addPerParticleParameter("z0")

for i, atom in enumerate(fixer.topology.atoms()):
    if atom.element.symbol != "H":
        atom_crd = fixer.positions[i]
        force.addParticle(i, atom_crd.value_in_unit(nanometers))

system.addForce(force)

In [ ]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(fixer.topology, system, integrator)
simulation.context.setPositions(fixer.positions)

In [ ]:
simulation.minimizeEnergy(maxIterations=100)

In [ ]:
minpositions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(fixer.topology, minpositions, open('/Users/takabak/Desktop/min.pdb', 'w'))

In [ ]:
simulation.reporters.append(PDBReporter('/Users/takabak/Desktop/output.pdb', 100))
simulation.reporters.append(StateDataReporter(stdout, 100, step=True, potentialEnergy=True, temperature=True))
simulation.step(1000)